### Modular Components for Mirage System Ablation Studies

This notebook provides **plug-and-play modules** for conducting various ablation studies on the Mirage system. To evaluate a particular variant, replace the corresponding vanilla component in base core script with the specialized component implementation supplied here.

For comprehensive guidance on designing and interpreting ablation experiments, refer to the relevant sections of the paper.

**Key points**

* The module APIs replicate those of the baseline code, enabling a straightforward drop-in workflow.
* Achieving optimal results typically requires:

  * a solid understanding of the prior work on which each component is based, and
  * careful hyperparameter tuning.
    *(The underlying GNN implementation is inherently stochastic; seeds can vary across execution run.)*

Use these modules to explore how existing methods interact with and potentially affect the Mirage system.

In [ ]:
def Add_benign_system_noise(attack_df, benign_df, benign_structure_count):
    unique_benign_procs = list(set(benign_df['actorID']))
    selected_benign_procs = unique_benign_procs[:benign_structure_count]
    selected_benign_df = benign_df[benign_df['actorID'].isin(selected_benign_procs)]

    combined_df = pd.concat([attack_df, selected_benign_df], ignore_index=True)
                      
    event_template = {'actorID': None, 'objectID': None, 'action': '', 'timestamp': '', 'exec': '', 'path': ''}
    
    for malicious_proc in GT_mal:
        for benign_proc in selected_benign_procs:
            new_event = event_template.copy()
            new_event['actorID'] = malicious_proc
            new_event['objectID'] = benign_proc
            combined_df = combined_df.append(new_event, ignore_index=True)
    
    return combined_df

In [ ]:
'''
FedProx client-side training loop used for our ablation study.

This function mirrors the API of `train_gnn_func`, allowing it to be
plug-and-played in any notebook of the base system:

1. Replace calls to `train_gnn_func` with `FedProx_model_training`.
2. Restart the notebook and run all cells with `training=True` to retrain
   the local GNN models under FedProx.
3. Execute the evaluation cells once training finishes to obtain results.

The workflow is identical across datasets; every notebook follows the same
code structure.
'''

def FedProx_model_training(nodes, labels, edges, mapp, pids, idx_to_pid, mu=0.1):
    
    global categories, epochs
    
    pid_to_gnn_index = map_pids_to_category_indices(pids, categories)
    
    set_pids = set(pids)
    proc_index = list(idx_to_pid.keys())

    train_splits = [[] for _ in range(len(categories))]
    
    for i in proc_index:
        pname = idx_to_pid[str(i)]
        split_indx = pid_to_gnn_index[pname]
        train_splits[split_indx].append(int(i))
        
    local_models = [copy.deepcopy(x) for x in templates]
    
    for i in range(len(local_models)):
        
        if len(train_splits[i]) == 0:
            local_models[i] = None
        else:
            global_model = None
            if f"global{i}.pth" in os.listdir("Content_FL_Exp"):
                global_model = copy.deepcopy(local_models[i])
                global_model.load_state_dict(torch.load(f"Content_FL_Exp/global{i}.pth"))

            optimizer = torch.optim.Adam(local_models[i].parameters(), lr=0.01, weight_decay=5e-4)
            criterion = CrossEntropyLoss()

            graph = Data(x=torch.tensor(nodes, dtype=torch.float).to(device),
                         y=torch.tensor(labels, dtype=torch.long).to(device),
                         edge_index=torch.tensor(edges, dtype=torch.long).to(device))
            
            mask = torch.tensor([False] * graph.num_nodes, dtype=torch.bool)
            mask[train_splits[i]] = True
            
            def get_neighbors(edge_index, nodes):
                neighbors = []
                for node in nodes:
                    mask = edge_index[0] == node
                    neighbors.extend(edge_index[1, mask].tolist())
                return torch.tensor(list(set(neighbors)), dtype=torch.long)

            one_hop_neighbors = get_neighbors(graph.edge_index, train_splits[i])
            two_hop_neighbors = get_neighbors(graph.edge_index, one_hop_neighbors)
            two_hop_neighbors = two_hop_neighbors[~mask[two_hop_neighbors]]
            mask[two_hop_neighbors] = True
            
            for epoch in range(epochs):
                print(f'Training GNN Category {i} Model for Epoch {epoch}')

                loader = NeighborLoader(graph, num_neighbors=[-1, -1], batch_size=5000, input_nodes=mask)
                total_loss = 0
                for subg in loader:
                    local_models[i].train()
                    optimizer.zero_grad() 
                    out = local_models[i](subg.x, subg.edge_index) 
                    loss = criterion(out, subg.y)
                    
                    if global_model:
                        prox_term = 0.0
                        for param, global_param in zip(local_models[i].parameters(), global_model.parameters()):
                            prox_term += (mu / 2) * torch.norm(param - global_param) ** 2
                        loss += prox_term
                    
                    loss.backward() 
                    optimizer.step()      
                    total_loss += loss.item() * subg.batch_size
                print("Loss: ", total_loss / mask.sum().item(), '\n')
    return local_models

In [ ]:
"""
FedOpt server-side aggregation loop used in our ablation study.

This routine mirrors the API of `server_aggregate`, so you can swap it
into any notebook of the base system.

Usage
-----
1. Replace calls to `server_aggregate` with `FedOpt_model_aggregation`.
2. Restart the notebook and run all cells with `training=True` to retrain
   the global models under FedOpt.
3. Run the evaluation cells once training completes to obtain results.

Because every notebook shares the same structure, the steps above work
for all datasets.
"""

def FedOpt_model_aggragation(all_global_models):
    global_models = [copy.deepcopy(x) for x in templates]
    for l in range(len(all_global_models)):
        clients = [m for m in all_global_models[l] if m is not None]
        if not clients:
            continue
        g = global_models[l]
        device = next(g.parameters()).device
        avg = {}
        with torch.no_grad():
            for k in g.state_dict().keys():
                avg[k] = torch.stack([c.state_dict()[k].to(device) for c in clients], 0).mean(0)
        global_opt[l].zero_grad()
        for name, p in g.named_parameters():
            p.grad = (p.data - avg[name]).detach()
        global_opt[l].step()
        torch.save(g.state_dict(), f"Content_FL_Exp/global{l}.pth")
    return global_models

In [ ]:
def Model_poisoning_simulation_with_multikrum_averaging(all_models, num_poisoned_clients=1, f=1, noise_scale=0.5):
    global_models = templates
    for l in range(len(all_models)):
        current = all_models[l]
        valid = [i for i, m in enumerate(current) if m is not None]
        n = len(valid)
        if n == 0:
            continue
        if num_poisoned_clients:
            poison = np.random.choice(valid, min(num_poisoned_clients, n), replace=False)
        else:
            poison = []
        for idx in poison:
            for p in current[idx].parameters():
                p.data += torch.randn_like(p) * noise_scale
        if n < 2 * f + 3:
            avg_state = {}
            for k in current[valid[0]].state_dict().keys():
                avg_state[k] = torch.stack([current[i].state_dict()[k] for i in valid]).mean(0)
            global_models[l].load_state_dict(avg_state)
            torch.save(global_models[l].state_dict(), f"Content_FL_Exp/global{l}.pth")
            continue
        g_state = global_models[l].state_dict()
        updates = []
        for i in valid:
            flat = []
            c_state = current[i].state_dict()
            for k in g_state.keys():
                flat.append((c_state[k] - g_state[k]).flatten())
            updates.append(torch.cat(flat))
        updates = torch.stack(updates)
        dist = torch.cdist(updates, updates, p=2).pow(2)
        mkr = n - f - 2
        scores = []
        for i in range(n):
            s, _ = torch.sort(dist[i])
            scores.append(s[1:mkr + 1].sum())
        scores = torch.tensor(scores)
        m = mkr
        sel = scores.topk(m, largest=False).indices
        avg_update = updates[sel].mean(0)
        ptr = 0
        new_state = {}
        for k in g_state.keys():
            numel = g_state[k].numel()
            new_state[k] = g_state[k] + avg_update[ptr:ptr + numel].view_as(g_state[k])
            ptr += numel
        global_models[l].load_state_dict(new_state)
        torch.save(global_models[l].state_dict(), f"Content_FL_Exp/global{l}.pth")
    return global_models


In [ ]:
def Model_poisoning_with_fed_averaging(all_models, num_poisoned_clients=1, noise_scale=0.1):
    global_models = copy.deepcopy(templates)
    client_indices = [(l, i) for l, layer in enumerate(all_models) for i, m in enumerate(layer) if m is not None]
    if not client_indices:
        return global_models
    k = min(num_poisoned_clients, len(client_indices))
    poisoned_clients = set(tuple(client_indices[i]) for i in np.random.choice(len(client_indices), k=k, replace=False))
    for l, layer in enumerate(all_models):
        if all(m is None for m in layer):
            continue
        global_state = global_models[l].state_dict()
        running_sum = {k: torch.zeros_like(v, dtype=torch.float32) for k, v in global_state.items()}
        count = 0
        for i, client_model in enumerate(layer):
            if client_model is None:
                continue
            client_sd = client_model.state_dict()
            is_poisoned = (l, i) in poisoned_clients
            for p_key in global_state.keys():
                param = client_sd[p_key].float()
                if is_poisoned:
                    param = param + torch.randn_like(param) * noise_scale
                running_sum[p_key] += param
            count += 1
        for p_key in global_state.keys():
            global_state[p_key] = (running_sum[p_key] / count).type(global_state[p_key].dtype)
        global_models[l].load_state_dict(global_state)
        torch.save(global_state, f"Content_FL_Exp/global{l}.pth")
    return global_models


In [ ]:
def model_aggregation_with_differential_privacy(client_models, epsilon=0.1, delta=0.01, C=1.0):
    global_model = copy.deepcopy(template)
    global_sd = global_model.state_dict()
    num_clients = len(client_models)
    sigma = (C / num_clients) * np.sqrt(2 * np.log(1.25 / delta)) / epsilon
    clipped_updates = []
    for m in client_models:
        upd = {k: (m.state_dict()[k] - global_sd[k]).float() for k in global_sd}
        flat = torch.cat([v.view(-1) for v in upd.values()])
        coeff = min(1.0, C / (torch.norm(flat, p=2) + 1e-12))
        for k in upd:
            upd[k] *= coeff
        clipped_updates.append(upd)
    for k in global_sd:
        stacked = torch.stack([u[k] for u in clipped_updates])
        mean_update = stacked.mean(0)
        noise = torch.randn_like(mean_update) * sigma
        global_sd[k] += (mean_update + noise).type(global_sd[k].dtype)
    global_model.load_state_dict(global_sd)
    torch.save(global_sd, "Content_FL_Exp/global.pth")
    return global_model